# Testing ChatESG

In [1]:
import requests
import json

In [2]:
TEST_UID = "#test-uid"
MODE = "local"

if MODE=="local":
    base_url = 'http://localhost:8000'

# Health check
requests.get(f"{base_url}/health").content, base_url

(b'{"status":"OK"}', 'http://localhost:8000')

### Chat

In [5]:
headers_chat = {
    "Content-Type": "application/json",
    "test-uid": TEST_UID
}

chat_payload = {
    "src": "skripsi",
    "model": "gpt-4o-mini",
    # "bsid": "c23aabb84ff24825be7d",
    "inputs": [
        # {"type": "text", "role": "user", "content": "hai apa itu esg, jelasin secara singkat"}
        # {"type": "text", "role": "user", "content": "sebutkan 5 jenis inisiatif esg"}
        {"type": "text", "role": "user", "content": "Berapakah total emisi gas rumah kaca langsung yang dihasilkan tahun 2023?"}
    ]
}

response = requests.post(f"{base_url}/api/v1/chat", headers=headers_chat, data=json.dumps(chat_payload), stream=True)
for line in response.iter_lines():
    if line:
        chunk = json.loads(line.decode('utf-8'))
        print("Chunk received:", chunk)
chunk

Chunk received: {'done': False, 'tools': ['Respon diterima'], 'responses': [], 'sources': None, 'bsid': 'c3521d7d4d0e414ca23e', 'bcid': 'b78cbffd-f773-4c6b-8028-13c2b3b91cec', 'metadata': None}
Chunk received: {'done': False, 'tools': ['Respon diterima'], 'responses': [''], 'sources': None, 'bsid': 'c3521d7d4d0e414ca23e', 'bcid': 'b78cbffd-f773-4c6b-8028-13c2b3b91cec', 'metadata': None}
Chunk received: {'done': False, 'tools': ['Respon diterima'], 'responses': ['Total'], 'sources': None, 'bsid': 'c3521d7d4d0e414ca23e', 'bcid': 'b78cbffd-f773-4c6b-8028-13c2b3b91cec', 'metadata': None}
Chunk received: {'done': False, 'tools': ['Respon diterima'], 'responses': ['Total em'], 'sources': None, 'bsid': 'c3521d7d4d0e414ca23e', 'bcid': 'b78cbffd-f773-4c6b-8028-13c2b3b91cec', 'metadata': None}
Chunk received: {'done': False, 'tools': ['Respon diterima'], 'responses': ['Total emisi'], 'sources': None, 'bsid': 'c3521d7d4d0e414ca23e', 'bcid': 'b78cbffd-f773-4c6b-8028-13c2b3b91cec', 'metadata': None

{'done': True,
 'tools': ['Respon diterima'],
 'responses': ['Total emisi gas rumah kaca langsung (Scope 1) yang dihasilkan oleh PT Astra International Tbk pada tahun 2023 adalah sebesar 4,604,076 ton CO2-eq.'],
 'sources': [{'url': 'https://example.com',
   'pages': [1, 2, 3],
   'snippet': 'this is snippet...'}],
 'bsid': 'c3521d7d4d0e414ca23e',
 'bcid': 'b78cbffd-f773-4c6b-8028-13c2b3b91cec',
 'metadata': None}